# Parameterization for sediment released by sea-ice

In [1]:
import numpy as np
import glob
import datetime as dt
import scipy.ndimage as ndimage
import xarray as xr
import sys
sys.path.append('../paper-materials/')
from constants import imin, imax, jmin, jmax

%matplotlib inline

#### Load files

In [4]:
# The Pb model configuration ANHA12 mask:
mesh      = xr.open_dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20230213.nc')
mesh_lon  = mesh['nav_lon'].values
mesh_lat  = mesh['nav_lat'].values
tmask     = mesh['tmask'].values
land_mask = np.ma.masked_where((tmask[0,:,:,:] > 0.1), tmask[0,:,:,:])

#### Functions:

In [5]:
# load trajectories from particle tracking experiments
def load_tracks(filename):
    nemo_file  = xr.open_dataset(filename)

    traj = nemo_file['trajectory'].values # dimensions: number of particles, tracks
    time = nemo_file['time'].values       # units: seconds
    lat  = nemo_file['lat'].values        # degrees North
    lon  = nemo_file['lon'].values        # degrees East

    return traj, time, lon, lat

In [38]:
# Check whether a parcel was in the laptev sea in the fall months
def check_laptev(CB_traj, CB_lon, CB_lat, CB_time):
    # Define boundary latitudes and longitudes for the Laptev Sea region
    trajS_bdy1 = 68;     trajN_bdy1 = 74;
    trajE_bdy1 = -170;   trajW_bdy1 = -210;

    trajS_bdy2 = 70;     trajN_bdy2 = 75;
    trajE_bdy2 = -185;   trajW_bdy2 = -230;
    
    Laptev_particle = False
    # At each time step:
    for timestep in range(0,len(CB_traj)):
        if ((CB_lon[timestep]   < trajE_bdy1) & (CB_lon[timestep] > trajW_bdy1) \
            & (CB_lat[timestep] < trajN_bdy1) & (CB_lat[timestep] > trajS_bdy1)) or \
           ((CB_lon[timestep]   < trajE_bdy2) & (CB_lon[timestep] > trajW_bdy2) \
            & (CB_lat[timestep] < trajN_bdy2) & (CB_lat[timestep] > trajS_bdy2)):

            start_time   = dt.datetime(2015,12,31) - dt.timedelta(seconds=CB_time[0].astype('timedelta64[s]').item().total_seconds())
            current_time = start_time - dt.timedelta(seconds=CB_time[timestep].astype('timedelta64[s]').item().total_seconds())

            # And is the parcel on the shelf in the fall?
            if current_time.month in [9,10,11,12]:
                Laptev_particle = True
                break
                    
    return Laptev_particle

In [39]:
# identify where a parcel came from
def parcel_origin(CB_lon, CB_lat, CB_time, CB_traj):

    dim_parc = int((CB_lon.shape[0]/12)/np.ceil(CB_lon.shape[1]/(4*365))) # bottom converts 6 hour to days    
    dim_time = int(12*((CB_lon.shape[0]/dim_parc)/12))

    particles_origin = np.zeros((dim_parc,dim_time))

    for release_time in range(0,dim_time):
        for location in range(0,dim_parc):
            ind = location + release_time*dim_parc
            lon_loc = CB_lon[ind,:]
            lat_loc = CB_lat[ind,:]
            time_loc = CB_time[ind,:]
            traj_loc = CB_traj[ind,:]

            Laptev_particle = check_laptev(traj_loc, lon_loc, lat_loc, time_loc)

            if Laptev_particle:
                particles_origin[location, release_time] = 1

    return particles_origin

In [40]:
def interp_np(nav_lon, nav_lat, var_in, lon_ANHA12, lat_ANHA12):
    ''' Interpolate some field to ANHA12 grid.
        The function is based on the bilinear interpolation in scipy, griddata 
        =======================================================================
            nav_lon, nav_lat        : input field lon/lat
            lon_ANHA12, lat_ANHA12  : ANHA12 defined lons/lats
            var_in                  : 2-D model variable
    '''
    from scipy.interpolate import griddata
    LatLonPair = (nav_lon, nav_lat)
    var_out = griddata(LatLonPair, var_in, (lon_ANHA12, lat_ANHA12), method='cubic')
    # Take nearest neighbour interpolation to fill nans
    var_fill = griddata(LatLonPair, var_in, (lon_ANHA12, lat_ANHA12), method='nearest')
    
    # fill nans with constant value (0.1)
    var_out[np.isnan(var_out)] = var_fill[np.isnan(var_out)]
    return var_out

Load parcel trajectories

In [41]:
# from simulations ran for Rogalla et al. (2022).
CB_traj, CB_time, CB_lon, CB_lat = load_tracks('/ocean/brogalla/GEOTRACES/parcels/trials/'+\
                                               'Particles_CB-20200205-extended-region2.nc')

particles_origin = parcel_origin(CB_lon, CB_lat, CB_time, CB_traj)

dim_parc = int((CB_lon.shape[0]/12)/np.ceil(CB_lon.shape[1]/(4*365)))
dim_lons = len(set(CB_lon[0:dim_parc,0]))

proportion_laptev = np.empty(CB_lon[0:dim_parc,0].shape)
# calculate the proportion that spent time in the Laptev Sea
for location in range(0,dim_parc):
    proportion_laptev[location] = np.sum(particles_origin[location,:])/particles_origin.shape[1]

In [42]:
parcel_lons = CB_lon[0:186, 0]
parcel_lats = CB_lat[0:186, 0]

Forcing field dimensions

In [43]:
forcing_lons = mesh_lon[:,:]
forcing_lats = mesh_lat[:,:]
forcing_sed  = np.zeros(forcing_lons.shape)

Interpolate Canada Basin proportions:

In [44]:
forcing_sed = interp_np(parcel_lons, parcel_lats, proportion_laptev, forcing_lons, forcing_lats)

In [45]:
forcing_sed[forcing_sed < 0] = 0

# North of Nares Strait
forcing_sed[(forcing_lons < -50) & (forcing_lons > -95) & (forcing_lats > 78) & (forcing_lats < 83.5)] = 0.03

# CAA background rate
forcing_sed[(forcing_lons >-128) & (forcing_lons < -45) & (forcing_lats < 77) & (forcing_lats > 60)] = 0.03

# Beaufort Shelf background rate
forcing_sed[(forcing_lons <-128) & (forcing_lats < 71.3) & (forcing_lats > 68)] = 0.02

# zero southern Baffin Bay
forcing_sed[(forcing_lons >-68)  & (forcing_lats<71)] = 0
forcing_sed[(forcing_lons >-100) & (forcing_lats<66)] = 0

In [46]:
Z2 = ndimage.gaussian_filter(forcing_sed, sigma=16, order=0)

# Zero the forcing field outside of the domain:
Z2[0:imin, :]  = 0; Z2[imax:-1, :] = 0;
Z2[:, 0:jmin]  = 0; Z2[:, jmax:-1] = 0;

#### Save to forcing field:

In [23]:
file_write = xr.Dataset(
    {'prop_shelf': (("y","x"), Z2)}, 
    coords = {
        "y": np.zeros(2400),
        "x": np.zeros(1632),
    },
    attrs = {
        'long_name':'Proportion of shelf sediments in ice',
        'units':'none',
    }
)

file_write.to_netcdf('/ocean/brogalla/GEOTRACES/data/ice_sediment-20221102.nc')